# Demo of the Models on Differential Equations

In [1]:
import os

import numpy
import pandas
import sympy
import torch

from src.envs import build_env
from src.envs.sympy_utils import simplify_equa_diff
from src.envs.char_sp import InvalidPrefixExpression
from src.envs.sympy_utils import simplify
from src.model import build_modules
from src.utils import AttrDict
from src.utils import to_cuda

Use the Model to Find the solutions to first and second order differential equations.

## First Order Differential Equations (ODE 1)

Procedure:
1. Start from a bivariate function $F(x,c)$, that will be the equation solution, that can be solved in $c$.
2. Solve $F(x,c)$ in $c$.
3. Differentiate in $x$.
4. Simplify the final form.

### Build Environment - Reload Model

Get Trained Model:

In [2]:
model_path = '../models/differential-equations/ode1.pth'
assert os.path.isfile(model_path)

Set the Parameters for Environment and for the Model:

 Environment:
- **env_name**: SymPy character environment.
- **int_base**: integer representation base.
- **balanced**: balanced representation (base > 0).
- **positive**: do not sample negative numbers.
- **precision**: float numbers precision.
- **n_variables**: number of variables in expressions (between 1 and 4).
- **n_coefficients**: number of coefficients in expressions (between 0 and 10).
- **leaf_probs**: leaf probabilities of being a variable, a coefficient, an integer, or a constant.
- **max_len**: maximum sequences length.
- **max_int**: maximum integer value.
- **max_ops**: maximum number of operators.
- **max_ops_G**: maximum number of operators for G in IBP.
- **clean_prefix_expr**: clean prefix expressions (f x -> Y, derivative f x x -> Y').
- **rewrite_functions**: rewrite expressions with a given SymPy function.
- **tasks**: tasks to run (prim_fwd, prim_bwd, prim_ibp, ode1, ode2).
- **operators**: considered operators (add, sub, mul, div), followed by (unnormalized) sampling probabilities.


 Model:
- **cpu**: run on CPU.
- **emb_dim**: embedding layer size.
- **n_enc_layers**: number of transformer layers in the encoder.
- **n_dec_layers**: number of transformer layers in the decoder.
- **n_heads**: number of transformer heads.
- **dropout**: dropout.
- **attention_dropout**: dropout in the attention layer.
- **sinusoidal_embeddings**: use sinusoidal embeddings.
- **share_inout_emb**: share input and output embeddings.
- **reload_model**: reload a pretrained model.

In [3]:
params = AttrDict({

    # Environment Parameters
    'env_name': 'char_sp',
    'int_base': 10,
    'balanced': False,
    'positive': True,
    'precision': 10,
    'n_variables': 1,
    'n_coefficients': 0,
    'leaf_probs': '0.75,0,0.25,0',
    'max_len': 512,
    'max_int': 5,
    'max_ops': 15,
    'max_ops_G': 15,
    'clean_prefix_expr': True,
    'rewrite_functions': '',
    'tasks': 'ode1',
    'operators': 'add:10,sub:3,mul:10,div:5,sqrt:4,pow2:4,pow3:2,pow4:1,pow5:1,ln:4,exp:4,sin:4,cos:4,tan:4,asin:1,'
                 'acos:1,atan:1,sinh:1,cosh:1,tanh:1,asinh:1,acosh:1,atanh:1',

    # Model Parameters
    'cpu': False,
    'emb_dim': 1024,
    'n_enc_layers': 6,
    'n_dec_layers': 6,
    'n_heads': 8,
    'dropout': 0,
    'attention_dropout': 0,
    'sinusoidal_embeddings': False,
    'share_inout_emb': True,
    'reload_model': model_path,

})

Set the Environment with SymPy:

In [4]:
env = build_env(params)
x = env.local_dict['x']
f = env.local_dict['f']
c = sympy.Symbol('c')

The primary components of the model are one encoder and one decoder network. The encoder turns each item into a
corresponding hidden vector containing the item and its context. The decoder reverses the process, turning the vector
into an output item, using the previous output as the input context.

Build Model Modules:

In [ ]:
modules = build_modules(env, params)
encoder = modules['encoder']
decoder = modules['decoder']

### Declare Differential Equation Input and its Solution

Declare a bivariate function $F(x,c)$, that will be the equation solution, that can be solved in c:

In [ ]:
# y_infix = 'x*log(c/x)'
# y_infix = '((4+(2*x))**(-1))*((x+(sin(x)))*((x**4)+(c*(x**(-1)))))'
# y_infix = 'exp(c+((sqrt(cos(x)))+(acos(2*x))))'
y_infix = '3+((c*(x**(-1)))+(sin(tanh(cos(x)))))'

Converts **y_infix** to a type that can be used inside SymPy:

In [ ]:
y = sympy.sympify(y_infix, locals=env.local_dict)
y

Solve $y$ in $c$:

In [ ]:
solve_c = sympy.solve(f(x) - y, c, check=False, simplify=False)
c = solve_c[0]
c

Differentiate $c$ in $x$:

In [ ]:
eq = c.diff(x)
eq = simplify(eq, seconds=1)
eq

Simplify previous differential equation:

In [ ]:
eq = simplify_equa_diff(eq, required=f(x).diff(x))
eq

### Compute Prefix Representations

In [ ]:
y_prefix = env.sympy_to_prefix(y)
eq_prefix = env.sympy_to_prefix(eq)
print(f"Solution y with Prefix Notation:\n{y_prefix}\n")
print(f"Differential Equation with Prefix Notation:\n{eq_prefix}")

### Encode Input

Clean prefix expressions before they are converted to PyTorch data.

Examples:
- f x  -> Y
- derivative f x x  -> Y'

In [ ]:
x1_prefix = env.clean_prefix(eq_prefix)
print(f"Differential Equation Clean Prefix Notation:\n{x1_prefix}")

Create a PyTorch LongTensor for storing $eq$ as a sequence of indexes based on prefix clean notation "words" (Word to
index dictionary is defined inside the Model environment):

In [ ]:
x1 = torch.LongTensor(
    [env.eos_index] +
    [env.word2id[w] for w in x1_prefix] +
    [env.eos_index]
).view(-1, 1)
x1.transpose(0, 1)

Move PyTorch tensors to CUDA (GPU):

In [ ]:
len1 = torch.LongTensor([len(x1)])
x1, len1 = to_cuda(x1, len1)

Encodes the “meaning” of the input sequence into a single vector, with the Encoder of the Model:

In [ ]:
with torch.no_grad():
    encoded = encoder('fwd', x=x1, lengths=len1, causal=False).transpose(0, 1)

encoded

### Decode with Beam Search

Instead of picking a single output, a sequence (in this case an hypothesis of differential equation solution), multiple
highly probable choices are retained.

Declare beam size:

In [ ]:
beam_size = 10

Takes the encoder output vector and outputs multiple sequences of "words", that in this case should represent the
solution $y$ for the differential equation $eq$, using the Decoder of the model.

In [ ]:
with torch.no_grad():
    _, _, beam = decoder.generate_beam(encoded, len1, beam_size=beam_size, length_penalty=1.0, early_stopping=1,
                                       max_len=params.max_len)
assert len(beam) == 1
hypotheses = beam[0].hyp
assert len(hypotheses) == beam_size

### View the Results

Input differential equation $eq$:

In [ ]:
eq


Solution $y$ to find:

In [ ]:
y

Extract scores and solution hypotheses:

In [ ]:
rows = numpy.arange(1, beam_size + 1)
columns = ['Score', 'Solution Hypothesis', 'Valid']
results = []

for score, sequence in sorted(hypotheses, reverse=True):
    # Parse decoded hypothesis
    ids = sequence[1:].tolist()  # Decoded token IDs
    hyp_prefix = [env.id2word[word_id] for word_id in ids]  # Convert to prefix notation

    try:
        hyp_infix = env.prefix_to_infix(hyp_prefix)  # Convert to infix notation
        hyp_sympy = env.infix_to_sympy(hyp_infix)  # Convert to SymPy

        # Check if the hypothesis is a valid solution, replacing 'hyp_sympy' with 'f(x)' in the equation
        validation = "YES" if simplify(eq.subs(f(x), hyp_sympy).doit(), seconds=1) == 0 else "NO"

        # Transform hypothesis to a valid latex expression
        hyp_expr = "$" + sympy.latex(hyp_sympy)  + "$"

    except InvalidPrefixExpression:
        validation = "INVALID PREFIX EXPRESSION"
        hyp_expr = hyp_prefix

    # Prepare results
    results.append([score, hyp_expr, validation])

Print results:

In [ ]:
pandas.set_option('max_colwidth', None)
pandas.DataFrame(results, index=rows, columns=columns).style.set_properties(**{'text-align': 'center'})

## Second Order Differential Equations (ODE 2)

Procedure:
1. Start from a trivariate function $F(x,c_1,c_2)$, that will be the equation solution, that can be solved in $c_2$.
2. Solve $F(x,c_1,c_2)$ in $c_2$.
3. Differentiate in $x$.
4. Solve in $c_1$.
5. Differentiate in $x$.
6. Simplify the final form.

### Build Environment - Reload Model

Get Trained Model:

In [ ]:
model_path = '../models/differential-equations/ode2.pth'
assert os.path.isfile(model_path)

params = AttrDict({

    # Environment Parameters
    'env_name': 'char_sp',
    'int_base': 10,
    'balanced': False,
    'positive': True,
    'precision': 10,
    'n_variables': 1,
    'n_coefficients': 0,
    'leaf_probs': '0.75,0,0.25,0',
    'max_len': 512,
    'max_int': 5,
    'max_ops': 15,
    'max_ops_G': 15,
    'clean_prefix_expr': True,
    'rewrite_functions': '',
    'tasks': 'ode1',
    'operators': 'add:10,sub:3,mul:10,div:5,sqrt:4,pow2:4,pow3:2,pow4:1,pow5:1,ln:4,exp:4,sin:4,cos:4,tan:4,asin:1,'
                 'acos:1,atan:1,sinh:1,cosh:1,tanh:1,asinh:1,acosh:1,atanh:1',

    # Model Parameters
    'cpu': False,
    'emb_dim': 1024,
    'n_enc_layers': 6,
    'n_dec_layers': 6,
    'n_heads': 8,
    'dropout': 0,
    'attention_dropout': 0,
    'sinusoidal_embeddings': False,
    'share_inout_emb': True,
    'reload_model': model_path,

})

env = build_env(params)
c1 = sympy.Symbol('c1')
c2 = sympy.Symbol('c2')

modules = build_modules(env, params)
encoder = modules['encoder']
decoder = modules['decoder']

### Declare Differential Equation Input and its Solution

Declare a bivariate function $F(x,c_1, c_2)$, that will be the equation solution, that can be solved in $c_2$:

In [ ]:
# y_infix = 'c1*exp(x)+c2*exp(-x)'
# y_infix = '(2*x)+((x*(c2+(c1*x)))+(exp(3)))'
# y_infix = '(x**9)*((c1+(c2*((cos(cosh(asin(tanh(x)))))**(-1))))**(-1))'
y_infix = 'c2*(5+((c1+(x+(x**3)))*(exp((-1)*((tan(x))**2)))))'

Converts **y_infix** to a type that can be used inside SymPy:

In [ ]:
y = sympy.sympify(y_infix, locals=env.local_dict)
y

Solve $y$ in $c_2$:

In [ ]:
solve_c2 = sympy.solve(f(x) - y, c2, check=False, simplify=False)
c2 = solve_c2[0]
c2

Differentiate $c_2$ in $x$:

In [ ]:
eq = c2.diff(x)
eq = simplify(eq, seconds=1)
eq

Solve $eq$ in $c_1$:

In [ ]:
solve_c1 = sympy.solve(eq, c1, check=False, simplify=False)
c1 = solve_c1[0]
c1

Differentiate $a_8$ in $x$:

In [ ]:
eq = c1.diff(x)
eq = simplify(eq, seconds=1)
eq

Simplify previous differential equation:

In [ ]:
eq = simplify_equa_diff(eq, required=f(x).diff(x, 2))
eq

### Compute Prefix Representations

In [ ]:
y_prefix = env.sympy_to_prefix(y)
eq_prefix = env.sympy_to_prefix(eq)
print(f"Solution y with Prefix Notation:\n{y_prefix}\n")
print(f"Differential Equation with Prefix Notation:\n{eq_prefix}")

### Encode Input

In [ ]:
x1_prefix = env.clean_prefix(eq_prefix)
print(f"Differential Equation Clean Prefix Notation:\n{x1_prefix}")

In [ ]:
x1 = torch.LongTensor(
    [env.eos_index] +
    [env.word2id[w] for w in x1_prefix] +
    [env.eos_index]
).view(-1, 1)

In [ ]:
len1 = torch.LongTensor([len(x1)])
x1, len1 = to_cuda(x1, len1)

In [ ]:
with torch.no_grad():
    encoded = encoder('fwd', x=x1, lengths=len1, causal=False).transpose(0, 1)

encoded

### Decode with Beam Search

In [ ]:
beam_size = 10

with torch.no_grad():
    _, _, beam = decoder.generate_beam(encoded, len1, beam_size=beam_size, length_penalty=1.0, early_stopping=1,
                                       max_len=params.max_len)
assert len(beam) == 1
hypotheses = beam[0].hyp
assert len(hypotheses) == beam_size

### View the Results

Input differential equation $eq$:

In [ ]:
eq

Solution $y$ to find:

In [ ]:
y

Extract scores and solution hypotheses:

In [ ]:
rows = numpy.arange(1, beam_size + 1)
columns = ['Score', 'Solution Hypothesis', 'Valid']
results = []

for score, sequence in sorted(hypotheses, reverse=True):
    # Parse decoded hypothesis
    ids = sequence[1:].tolist()  # Decoded token IDs
    hyp_prefix = [env.id2word[word_id] for word_id in ids]  # Convert to prefix notation

    try:
        hyp_infix = env.prefix_to_infix(hyp_prefix)  # Convert to infix notation
        hyp_sympy = env.infix_to_sympy(hyp_infix)  # Convert to SymPy

        # Check if the hypothesis is a valid solution, replacing 'hyp_sympy' with 'f(x)' in the equation
        validation = "YES" if simplify(eq.subs(f(x), hyp_sympy).doit(), seconds=1) == 0 else "NO"

        # Transform hypothesis to a valid latex expression
        hyp_expr = "$" + sympy.latex(hyp_sympy)  + "$"

    except InvalidPrefixExpression:
        validation = "INVALID PREFIX EXPRESSION"
        hyp_expr = hyp_prefix

    # Prepare results
    results.append([score, hyp_expr, validation])

Print results:

In [ ]:
pandas.set_option('max_colwidth', None)
pandas.DataFrame(results, index=rows, columns=columns).style.set_properties(**{'text-align': 'center'})